Importing necessary Libraries 

Libray version in my computer

Package                            Version
---------------------------------- --------------------
anaconda-client                    1.9.0

anaconda-navigator                 2.1.1

anaconda-project                   0.10.1

keras                              2.9.0

Keras-Preprocessing                1.1.2

nltk                               3.6.5

pandas                             1.3.4

pip                                21.2.4

scikit-image                       0.18.3

scikit-learn                       0.24.2

scipy                              1.7.1

spacy                              3.3.0

tensorflow                         2.9.1

tensorflow-estimator               2.9.0

tensorflow-gpu                     2.9.0

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from keras_preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from scipy.sparse import csr_matrix
from nltk.corpus import stopwords
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score

Reading the Data from the file 

In [2]:
read_data = open('training.data', 'r+')

In [3]:
read_data

<_io.TextIOWrapper name='training.data' mode='r+' encoding='cp1252'>

In [4]:
df= pd.DataFrame(read_data.readlines(), columns = ['Question'])

In [5]:
print(df)

                                                Question
0      DESC:manner How did serfdom develop in and the...
1      ENTY:cremat What films featured the character ...
2      DESC:manner How can I find a list of celebriti...
3      ENTY:animal What fowl grabs the spotlight afte...
4             ABBR:exp What is the full form of .com ?\n
...                                                  ...
15447  ENTY:other What 's the shape of a camel 's spi...
15448  ENTY:currency What type of currency is used in...
15449         NUM:temp What is the temperature today ?\n
15450   NUM:temp What is the temperature for cooking ?\n
15451  ENTY:currency What currency is used in Austral...

[15452 rows x 1 columns]


Creating the above data to data frame for classification. Creating the data as:
1. The question after the first space of the classifiers 
2. Spliting the classifier and making as two classifer

In [6]:
df['C'] = df.Question.apply(lambda x: x.split(' ', 1)[0])
df['Question'] = df.Question.apply(lambda x: x.split(' ', 1)[1])
df['C1'] = df.C.apply(lambda x: x.split(':')[0])
df['C2'] = df.C.apply(lambda x: x.split(':')[1])
df=df.drop(columns=['C'])

In [7]:
df.head(10)

,Question,C1,C2
0,How did serfdom develop in and then leave Russ...,DESC,manner
1,What films featured the character Popeye Doyle...,ENTY,cremat
2,How can I find a list of celebrities ' real na...,DESC,manner
3,What fowl grabs the spotlight after the Chines...,ENTY,animal
4,What is the full form of .com ?\n,ABBR,exp
5,What contemptible scoundrel stole the cork fro...,HUM,ind
6,What team did baseball 's St. Louis Browns bec...,HUM,gr
7,What is the oldest profession ?\n,HUM,title
8,What are liver enzymes ?\n,DESC,def
9,Name the scar-faced bounty hunter of The Old W...,HUM,ind


Since we only have training data, so to verify splitting in training and testing data respectively. If there is a different testing data consider df_train to be df itself i.e., full data can be used for training.  

In [8]:
df_train=df[0:int(0.66*len(df))]

Labeling the two classifying variable

In [9]:
c1=df_train['C1']
le_c1=LabelEncoder()
le_c1.fit(c1)
Y_train_c1=le_c1.transform(c1)

In [10]:
print(Y_train_c1)

[1 2 1 ... 3 3 5]


In [11]:
c2=df_train['C2']
le_c2=LabelEncoder()
le_c2.fit(c2)
Y_train_c2=le_c2.transform(c2)

In [12]:
print(Y_train_c2)

[23  8 23 ... 19 19 10]


In [13]:
print(len(set(c1)))
print(len(set(c2)))

6
47


In [14]:
X_train=df_train['Question'].tolist()

In [15]:
print(X_train)

['How did serfdom develop in and then leave Russia ?\n', 'What films featured the character Popeye Doyle ?\n', "How can I find a list of celebrities ' real names ?\n", 'What fowl grabs the spotlight after the Chinese Year of the Monkey ?\n', 'What is the full form of .com ?\n', 'What contemptible scoundrel stole the cork from my lunch ?\n', "What team did baseball 's St. Louis Browns become ?\n", 'What is the oldest profession ?\n', 'What are liver enzymes ?\n', 'Name the scar-faced bounty hunter of The Old West .\n', 'When was Ozzy Osbourne born ?\n', 'Why do heavier objects travel downhill faster ?\n', 'Who was The Pride of the Yankees ?\n', 'Who killed Gandhi ?\n', 'What is considered the costliest disaster the insurance industry has ever faced ?\n', 'What sprawling U.S. state boasts the most airports ?\n', 'What did the only repealed amendment to the U.S. Constitution deal with ?\n', 'How many Jews were executed in concentration camps during WWII ?\n', "What is `` Nine Inch Nails '

In [16]:
punc_stop = '''!()-[]{};:'"`\,<>./?@#$%^&*_~\n'''

Function to remove Punctuation and stopwords from data to create the Vocabulary. Punctuation and stop words can safely removed since it provides less information and they don't help us to find the context or the true meaning of a sentence. 

In [17]:
def removepunc_stopwords(X):
    #removing the punctuation
    for i in range(len(X)):
        question = X[i].lower()
        no_punct = ''
        for char in question:
            if char not in punc_stop:
                no_punct = no_punct + char
        no_punct = no_punct.replace("  ", " ")
        X[i] = no_punct
    
    #do not consider this stop words as this words helps to classify the problem
    words = ['how', 'what','which', 'where', 'why', 'whom', 'when','who']
    stop = set(stopwords.words('english'))
    for word in words:
        stop.remove(word)
    X = [[x for x in x.split() if x not in stop] for x in X]
    X = [' '.join(x) for x in X]
    
    return X

In [18]:
X_train=removepunc_stopwords(X_train)
print(X_train[0:10])

['how serfdom develop leave russia', 'what films featured character popeye doyle', 'how find list celebrities real names', 'what fowl grabs spotlight chinese year monkey', 'what full form com', 'what contemptible scoundrel stole cork lunch', 'what team baseball st louis browns become', 'what oldest profession', 'what liver enzymes', 'name scarfaced bounty hunter old west']


In [19]:
print(Y_train_c1)
print(Y_train_c2)

[1 2 1 ... 3 3 5]
[23  8 23 ... 19 19 10]


Converting the Text into vector using Count Vectorizer for further compuation using X_train_1 to fit the data as it will also be used in convering the test data for having the same vocab. Also converting it into csr matrix as data consisits of zeros

In [20]:
vectorizer = CountVectorizer()
X_train_1 = vectorizer.fit(X_train)
X_train_final=X_train_1.transform(X_train)

In [21]:
X_train_final=csr_matrix(X_train_final)

In [22]:
X_train_final

<10198x7012 sparse matrix of type '<class 'numpy.int64'>'
	with 56702 stored elements in Compressed Sparse Row format>

Creating the model for classification using LinearSVC. Creating two models for tow classification reapectively.

In [23]:
from sklearn.svm import LinearSVC
model= LinearSVC(C=1.0,multi_class='ovr')
model_1= LinearSVC(C=1.0,multi_class='ovr')

X_train_final

<10198x7012 sparse matrix of type '<class 'numpy.int64'>'
	with 56702 stored elements in Compressed Sparse Row format>

In [24]:
model_c1=model.fit(X_train_final, Y_train_c1)

In [25]:
model_c2=model_1.fit(X_train_final,Y_train_c2)

Creating the test data. If there is a differernt testing data it can be used here. Also, after importing the testing data we have to convert it into the respective data frame. We can use cell 6 to do so.  

In [26]:
df_test=df[int(0.66*len(df)):]

Function to pre process the testing data. Removing spaces and stop words

In [27]:
def preprocess_testdata(df_test):
    X=df_test['Question'].tolist()
    X= removepunc_stopwords(X)
    
    return X

In [28]:
X_test=preprocess_testdata(df_test)

In [29]:
print(X_test)

['what countries largest areas forest', 'how start web based business', 'name alvin brothers', 'what eagle syndrome styloid process', 'how jayne mansfield die', 'when royal wedding prince andrew fergie take place', 'what actor said day races either dead watch stopped', 'what explorer nicknamed iberia pilot', 'cnn began broadcasting what year', 'what ball hits foul pole called', 'what makes hamburgers taste like steakburgers', 'which company created internet browser mosaic', 'what rites accompanying circumcision newly bornchild judaism called', 'what education system 1960', 'what south american city world highest commercial landing field', 'how find which cities cablemodem access', 'what betsy ross famous', 'how many species sharks', 'what snoogans', 'how many people world', 'what shrubs planted safe deer', 'what organization offices broken watergate 1972', 'what design ship titanic', 'what sos stand', 'name technique widely used detect birth defects', 'what daily requirement folic acid

Transforming the testing data using count vectorizer and  converting it into a sparse matrix representation due to multiple zeros in the data

In [30]:
X_test = X_train_1.transform(X_test)

In [31]:
X_test=csr_matrix(X_test)

In [32]:
X_test

<5254x7012 sparse matrix of type '<class 'numpy.int64'>'
	with 27910 stored elements in Compressed Sparse Row format>

Predict the model for Classifier 1

In [33]:
preds_c1 = model_c1.predict(X_test)

In [34]:
preds_c1

array([4, 1, 3, ..., 1, 5, 1])

Predict the model for classifier 2

In [35]:
preds_c2 = model_c2.predict(X_test)

In [36]:
preds_c2

array([ 7, 23, 19, ..., 11, 40, 11])